HOLD - OUT

In [21]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Função para normalização dos dados
def normalize_data(train, test):
    scaler = StandardScaler()
    scaler.fit(train)
    return scaler.transform(train), scaler.transform(test)

# Função para adicionar média móvel como nova feature
def add_moving_average(data, window_size=5):
    data[f'Moving_Average_{window_size}'] = data['Dif_Preco_Dolar'].rolling(window=window_size).mean().shift(1)
    return data

# Caminhos para os arquivos de dados
file_paths = ['C:/Users/milen/OneDrive/Documentos/TCC/Bases/euro.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/taxa_br.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/bolsa_eua.csv']

# Processamento e modelagem para cada arquivo
for file_path in file_paths:
    data = pd.read_csv(file_path)

    data['Data'] = pd.to_datetime(data['Data'])
    data = data.iloc[1:]

    numeric_columns = data.select_dtypes(include=[np.number]).columns.tolist()
    data = remove_outliers(data, numeric_columns)

    # Adicionando média móvel
    data = add_moving_average(data)

    train_data = data[data['Data'] < pd.to_datetime("2020-01-01")]
    test_data = data[data['Data'] >= pd.to_datetime("2020-01-01")]

    y_train = train_data['Dif_Preco_Dolar']
    X_train = train_data.drop(columns=['Data', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real', 'Dif_Preco_Dolar', f'Moving_Average_{5}'])

    y_test = test_data['Dif_Preco_Dolar']
    X_test = test_data.drop(columns=['Data', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real', 'Dif_Preco_Dolar', f'Moving_Average_{5}'])

    mean_values_train = X_train.mean()
    X_train.fillna(mean_values_train, inplace=True)

    mean_values_test = X_test.mean()
    X_test.fillna(mean_values_test, inplace=True)

    non_numeric_columns = X_train.select_dtypes(include=['object']).columns
    for col in non_numeric_columns:
        X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
        X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

    # Normalizando os dados
    X_train, X_test = normalize_data(X_train, X_test)

    model = LinearRegression()
    model.fit(X_train, y_train)

    # Fazendo previsões no conjunto de teste
    y_pred = model.predict(X_test)

    # Avaliando o modelo
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Resultados para {file_path}:")
    print("MSE:", mse)
    print("MAE:", mae)
    print("R^2:", r2)
    print()



Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/euro.csv:
MSE: 1.658165220618404
MAE: 1.0362071279651202
R^2: 0.4019930540826495

Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv:
MSE: 1.6433515413904782
MAE: 1.009759704419117
R^2: 0.40397079795078294

Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv:
MSE: 1.5402295250677163
MAE: 0.9969930762573955
R^2: 0.4097756918947779

Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/taxa_br.csv:
MSE: 2.8343447299565967
MAE: 1.3409243290744357
R^2: -0.009893289668700955

Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/bolsa_eua.csv:
MSE: 2.5757824516659205
MAE: 1.290238692041564
R^2: 0.003956159389508773



JANELA DESLIANTE D+1

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Função para criar janelas deslizantes
def create_sliding_windows(data, window_size):
    windows = []
    for i in range(len(data) - window_size + 1):
        window = data[i:i + window_size]
        windows.append(window)
    return np.array(windows)

# Função para normalização dos dados
def normalize_data(train, test):
    scaler = StandardScaler()
    scaler.fit(train)
    return scaler.transform(train), scaler.transform(test)

# Função para adicionar média móvel como nova feature
def add_moving_average(data, window_size=1):
    data[f'Moving_Average_{window_size}'] = data['Dif_Preco_Dolar'].rolling(window=window_size).mean().shift(1)
    return data

# Caminhos para os arquivos de dados
file_paths = ['C:/Users/milen/OneDrive/Documentos/TCC/Bases/euro.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/taxa_br.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/bolsa_eua.csv']

# Processamento e modelagem para cada arquivo
for file_path in file_paths:
    data = pd.read_csv(file_path)

    data['Data'] = pd.to_datetime(data['Data'])
    data = data.iloc[1:]

    numeric_columns = data.select_dtypes(include=[np.number]).columns.tolist()
    data = remove_outliers(data, numeric_columns)

    # Adicionando média móvel
    data = add_moving_average(data)

    train_data = data[data['Data'] < pd.to_datetime("2021-01-01")]
    test_data = data[data['Data'] >= pd.to_datetime("2021-01-01")]

    y_train = train_data['Dif_Preco_Dolar']
    X_train = train_data.drop(columns=['Data', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real', 'Dif_Preco_Dolar', f'Moving_Average_{1}'])

    y_test = test_data['Dif_Preco_Dolar']
    X_test = test_data.drop(columns=['Data', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real', 'Dif_Preco_Dolar', f'Moving_Average_{1}'])

    mean_values_train = X_train.mean()
    X_train.fillna(mean_values_train, inplace=True)

    mean_values_test = X_test.mean()
    X_test.fillna(mean_values_test, inplace=True)

    non_numeric_columns = X_train.select_dtypes(include=['object']).columns
    for col in non_numeric_columns:
        X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
        X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

    # Normalizando os dados
    X_train, X_test = normalize_data(X_train, X_test)

    model = LinearRegression()
    model.fit(X_train, y_train)

    # Definindo o tamanho da janela para a parte de teste
    window_size = 1  # Pode ajustar conforme necessário

    # Criando janelas deslizantes para os dados de teste
    test_windows = create_sliding_windows(X_test, window_size)

    # Fazendo previsões para cada janela
    predictions = []
    for window in test_windows:
        pred = model.predict(window)
        predictions.append(pred[-1])  # Considerando apenas a última previsão de cada janela

    # Ajustando o conjunto y_test para se alinhar com as previsões
    y_test_adjusted = y_test[window_size - 1:]

    # Avaliando o modelo
    mse = mean_squared_error(y_test_adjusted, predictions)
    mae = mean_absolute_error(y_test_adjusted, predictions)
    r2 = r2_score(y_test_adjusted, predictions)

    print(f"Resultados para {file_path}:")
    print("MSE:", mse)
    print("MAE:", mae)
    print("R^2:", r2)
    print()



Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/euro.csv:
MSE: 1.5569126150227417
MAE: 1.0028535029481147
R^2: 0.2989644727570302

Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv:
MSE: 1.4571189515692362
MAE: 0.949087926427392
R^2: 0.42419710149026435

Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv:
MSE: 1.5319782633051409
MAE: 0.9791373404989729
R^2: 0.41311798715883696

Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/taxa_br.csv:
MSE: 2.66608026104892
MAE: 1.3160384606422157
R^2: -0.018225615407113427

Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/bolsa_eua.csv:
MSE: 2.6324539548224712
MAE: 1.3249782337206746
R^2: -0.009046706879204836

Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/clima.csv:
MSE: 4.702770256650913
MAE: 1.6796866882647274
R^2: -0.7318655197737782



JANELA DESLISANTE 30+

In [20]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Função para criar janelas deslizantes
def create_sliding_windows(data, window_size):
    windows = []
    for i in range(len(data) - window_size + 1):
        window = data[i:i + window_size]
        windows.append(window)
    return np.array(windows)

# Função para normalização dos dados
def normalize_data(train, test):
    scaler = StandardScaler()
    scaler.fit(train)
    return scaler.transform(train), scaler.transform(test)

# Função para adicionar média móvel como nova feature
def add_moving_average(data, window_size=30):
    data[f'Moving_Average_{window_size}'] = data['Dif_Preco_Dolar'].rolling(window=window_size).mean().shift(1)
    return data

# Caminhos para os arquivos de dados
file_paths = ['C:/Users/milen/OneDrive/Documentos/TCC/Bases/euro.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/taxa_br.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/bolsa_eua.csv']

# Processamento e modelagem para cada arquivo
for file_path in file_paths:
    data = pd.read_csv(file_path)

    data['Data'] = pd.to_datetime(data['Data'])
    data = data.iloc[1:]

    numeric_columns = data.select_dtypes(include=[np.number]).columns.tolist()
    data = remove_outliers(data, numeric_columns)

    # Adicionando média móvel
    data = add_moving_average(data)

    train_data = data[data['Data'] < pd.to_datetime("2021-01-01")]
    test_data = data[data['Data'] >= pd.to_datetime("2021-01-01")]

    y_train = train_data['Dif_Preco_Dolar']
    X_train = train_data.drop(columns=['Data', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real', 'Dif_Preco_Dolar', f'Moving_Average_{30}'])

    y_test = test_data['Dif_Preco_Dolar']
    X_test = test_data.drop(columns=['Data', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real', 'Dif_Preco_Dolar', f'Moving_Average_{30}'])

    mean_values_train = X_train.mean()
    X_train.fillna(mean_values_train, inplace=True)

    mean_values_test = X_test.mean()
    X_test.fillna(mean_values_test, inplace=True)

    non_numeric_columns = X_train.select_dtypes(include=['object']).columns
    for col in non_numeric_columns:
        X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
        X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

    # Normalizando os dados
    X_train, X_test = normalize_data(X_train, X_test)

    model = LinearRegression()
    model.fit(X_train, y_train)

    # Definindo o tamanho da janela para a parte de teste
    window_size = 30  # Pode ajustar conforme necessário

    # Criando janelas deslizantes para os dados de teste
    test_windows = create_sliding_windows(X_test, window_size)

    # Fazendo previsões para cada janela
    predictions = []
    for window in test_windows:
        pred = model.predict(window)
        predictions.append(pred[-1])  # Considerando apenas a última previsão de cada janela

    # Ajustando o conjunto y_test para se alinhar com as previsões
    y_test_adjusted = y_test[window_size - 1:]

    # Avaliando o modelo
    mse = mean_squared_error(y_test_adjusted, predictions)
    mae = mean_absolute_error(y_test_adjusted, predictions)
    r2 = r2_score(y_test_adjusted, predictions)

    print(f"Resultados para {file_path}:")
    print("MSE:", mse)
    print("MAE:", mae)
    print("R^2:", r2)
    print()



Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/euro.csv:
MSE: 1.6820820123309908
MAE: 1.041119055625903
R^2: 0.26899096947350865

Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv:
MSE: 1.4985752096749874
MAE: 0.9655405710660706
R^2: 0.4174220388836216

Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv:
MSE: 1.4664931801495207
MAE: 0.9550834475842662
R^2: 0.4474350156233873

Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/taxa_br.csv:
MSE: 2.6210513329141305
MAE: 1.3212078488350438
R^2: -0.01856174640764685

Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/bolsa_eua.csv:
MSE: 2.544898316030557
MAE: 1.3075807660381809
R^2: -0.0052912201421808636

